# Pre Train 预训练
预训练阶段，目标是让模型学习一下整体的语言风格，能够随机输出一些类似风格的字符串。这个阶段还不期望模型能够理解各个特殊分隔符的含义以及元信息与诗词之间的关系。因此我们通过直接拼接诗词内容的方式生成一个连续的字符串序列作为训练数据，并使用比较简单的训练策略。

> 训练前模型生成的结果：“溍蝭缔，最惕銅甍最喈血。。最。喈惕，最溍溍惕锄。，最紿。锄锄”
>
> 训练后模型生成的结果：“何須更取一杯酒，不用從君作醉鄉。春入湖光滿眼明，江湖處處是清明。”

在 Notebook 中我我们简化了训练过程，目的是让代码更好理解且运行更快的同时也能观察到一些效果。

In [ ]:
import torch
import random

from nanopoet.common import CharTokenizer, BEGIN
from nanopoet.dataset import load_raw_data, split_data
from nanopoet.model import GPTLanguageModel

# 初始化随机种子，让重复执行的结果稳定
random.seed(12345)
torch.manual_seed(12345)

# 首先加载数据、设备信息
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
data = load_raw_data("../raw")

# 初始化分词器
tokenizer = CharTokenizer("".join(["".join(list(d.values())) for d in data]))

In [ ]:
# 初始化模型结构
block_size = 256
model = GPTLanguageModel(
    vocab_size=tokenizer.vocab_size,
    emb_size=256,
    block_size=block_size,
    layer_num=8,
    head_num=8,
    dropout=0.1,
).to(device)

In [1]:
# 构建 Pre Train 阶段使用的数据集，将诗词内容直接拼接成一段文本
train, val = split_data(data)
train_data = torch.tensor(tokenizer.encode("".join([d["content"] for d in train])), dtype=torch.long)
val_data = torch.tensor(tokenizer.encode("".join([d["content"] for d in val])), dtype=torch.long)

# 构建一个从所有数据里随机抽取一段作为训练内容的函数
def get_batch(data, batch_size, block_size, device):
    """数据加载函数"""
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

In [8]:
# 设置预训练配置
batch_size = 64
max_steps = 100
eval_interval = 10
eval_iters = 5
# 使用一个固定的学习率
learning_rate = 3e-4

# 初始化优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [4]:
# 简化的训练循环
for step in range(max_steps):
    xb, yb = get_batch(train_data, batch_size, block_size, device)
    # 前向传播
    logits, loss = model(xb, yb)
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 定期评估、打印Loss并生成一段测试样本
    if step % eval_interval == 0:
        model.eval()
        out = {}
        for split, data in [('train', train_data), ('val', val_data)]:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(data, batch_size, block_size, device)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        context = torch.tensor([tokenizer.encode(BEGIN)], dtype=torch.long, device=device)
        generated = model.generate(context, max_new_tokens=20)
        sample = tokenizer.decode(generated[0].tolist())
        model.train()
        print(f"Step {step:5d} | Train Loss: {out['train']:.4f} | Val Loss: {out['val']:.4f}")
        print("生成样本：", sample)

Step     0 | Train Loss: 9.1039 | Val Loss: 9.0944
生成样本： B麞菘耿亡絝園匕淘竞襯瑞殛蜎丁葠蔯驮鎒该絻
Step     1 | Train Loss: 8.9113 | Val Loss: 8.9164
生成样本： B筒鬐臄袨㗚勖蹂鮎涡䟫辨战鉺歟杖福伥迎雰藐
Step    10 | Train Loss: 8.1654 | Val Loss: 8.1762
生成样本： B塾跪篿吳漿袷橇侉䍽惷䮂薳伴珝縫臥贵剽允功
Step    20 | Train Loss: 7.4969 | Val Loss: 7.5013
生成样本： B倩展噤磯招詬繄㰅妺孥嵐况㰞化分袖纖慾念塤
Step    30 | Train Loss: 7.0953 | Val Loss: 7.0781
生成样本： B相摒一吹鳥半沬走雲塞苷落園愜塔虐枕家野鳝
Step    40 | Train Loss: 6.9285 | Val Loss: 6.8851
生成样本： B動燕顋君亦滏西可夜雲間弦修上宜广應碓隨穿
Step    50 | Train Loss: 6.8356 | Val Loss: 6.8361
生成样本： B吞不，雞霜硌無採蕭薶。快生江夢衾梢憑百柳
Step    60 | Train Loss: 6.8214 | Val Loss: 6.8103
生成样本： B鼓壓喜陽卿謎材歌心應迴微寐書携丹沈，當鄣
Step    70 | Train Loss: 6.7902 | Val Loss: 6.7858
生成样本： B是古枕舊日像從同。知秋助識盡殘崎一雪佐城
Step    80 | Train Loss: 6.7741 | Val Loss: 6.7896
生成样本： B閑列水连生沙偉歲目，輪裏力滄外如草花盡始
Step    90 | Train Loss: 6.7355 | Val Loss: 6.7463
生成样本： B黄如長蹤渔，擗擬。堪，未蠶，密何雲風携。


# 误差
这里我们来解释一下在这个项目下，误差数值上的含义。

1. 我们构建的模型是一个基于词表的语言模型，词表的大小约为 11K（在我们使用字符级分词器的情况下）。
2. 评估的损失函数使用的是交叉熵（Cross Entropy）
3. 每次预测都是在词表中进行随机猜测，而正确答案只有一个，那么 Loss 是ln(11868) ≈ 9.38
4. 假设训练使的Loss 降低到 3.1，那么说明模型在预测下一个字符时，“犹豫范围” 从 11K 缩小到了 e^3.1 ≈ 22 个字符。

因此，随着训练次数的增加，模型会逐渐学到更多的“诗词模式”，每次预测时“选择范围”会缩小，准确度有所提高。

In [7]:
from pathlib import Path

# 把训练结果保存下来，给后续训练使用
output_path = Path("./output/02_pre_train_model.pt")
output_path.parent.mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), output_path)
print(f"模型已保存到 {output_path}")


训练完成！最终模型已保存到 output/01_pre_train_model.pt
